In this case we will try to run more epochs in this model hoping that we could get a better result using this model.

In [38]:
#@title Configuration of the variables notebook
model_name = 'roberta-base' #@param
value_batch_size =16 #@param
value_learning_rate = 2.057626963810742e-06 #@param
value_warmup = 0.2410014240231825 #@param
value_w_decay = 8.747541196471078e-09 #@param
value_n_epochs = 4  #@param
value_dropout = 0.3789390401864389 #@param
value_n_labels = 6 #@param

config_models ={
    "model_name": model_name,
    "n_labels": value_n_labels,
    "batch_size": value_batch_size,
    "lr": value_learning_rate,
    "warmup": value_warmup,
    "train_size": None,
    "w_decay": value_w_decay,
    "n_epochs": value_n_epochs,
    "dropout_vals": value_dropout
}

# Preparation for the model

In [39]:
%%capture
!pip install pytorch_lightning bertviz

In [40]:
#@title Importing the modules
from google.colab import drive
import os
import torch
drive.mount('/content/drive')

enable_code_paths = True
enable_block = False

if enable_code_paths:
    URL_path = "/content/drive/MyDrive/Code/Data_science_Turing_college/Colabs/14_002_NLP/dataset/clean_dataset.csv"
    URL_synthetic_data = "/content/drive/MyDrive/Code/Data_science_Turing_college/Colabs/14_002_NLP/dataset/synthetic_data.csv"
    URL_path_save = "/content/drive/MyDrive/Code/Data_science_Turing_college/Colabs/14_002_NLP/model"
    URL_helper ="/content/drive/MyDrive/Code/Data_science_Turing_college/Colabs/14_002_NLP/helper/"
    URL_hyperparameters ="/content/drive/MyDrive/Code/Data_science_Turing_college/Colabs/14_002_NLP/hyperparameters/"
    URL_test_labels ="/content/drive/MyDrive/Code/Data_science_Turing_college/Colabs/14_002_NLP/dataset/test_labels.csv"
    URL_test ="/content/drive/MyDrive/Code/Data_science_Turing_college/Colabs/14_002_NLP/dataset/test.csv"
    URL_model_saved_015 ="/content/drive/MyDrive/Code/Data_science_Turing_college/Colabs/14_002_NLP/model/015_model"
    URL_model_saved_017 ="/content/drive/MyDrive/Code/Data_science_Turing_college/Colabs/14_002_NLP/model/017_model"
else :
    URL_path = "/content/drive/MyDrive/turing_college_experiments/14_NLP/helper"
    URL_path_save = "/content/drive/MyDrive/turing_college_experiments/14_NLP/model"
    URL_helper = "/content/drive/MyDrive/turing_college_experiments/14_NLP/helper"
import sys
sys.path.append(URL_helper)

from colab_helper import proportion_balance_classes

"""
Verify the cores and gpu in the colab
"""
num_cores = os.cpu_count()  # Get the number of CPU cores
print(f'Number of available CPU cores: {num_cores}')
num_workers_colab = num_cores - 1
print(f'Setting num_workers to: {num_workers_colab}')

num_gpus_colab = torch.cuda.device_count()
gpu_list = [torch.cuda.get_device_name(i) for i in range(num_gpus_colab)]
num_gpus_colab, gpu_list


"""
import the modules and configuration colors
"""
## Core Libraries
import os
import time
import math
import numpy as np
import pandas as pd

# Sklearn for Evaluation Metrics and Model Selection
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import (
    accuracy_score, recall_score, multilabel_confusion_matrix, ConfusionMatrixDisplay
)
from sklearn.model_selection import train_test_split

# PyTorch Libraries and Lightning
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import AdamW
import pytorch_lightning as pl

# Hugging Face Transformers
from transformers import (
    AutoTokenizer, AutoModel,
    get_linear_schedule_with_warmup, utils
)

# Visualization Libraries
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from bertviz import model_view

# Progress Bar
from tqdm import tqdm

# Hyperparameter Tuning
import joblib

# Google Colab (if applicable)
from google.colab import drive

# TorchMetrics (for AUROC calculation)
from torchmetrics.functional.classification import auroc

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# Suppress standard warnings
utils.logging.set_verbosity_error()

"""
Colorama
"""
custom_colors = ['#36CE8A', "#7436F5","#3736F4",   "#36AEF5", "#B336F5", "#f8165e", "#36709A",  "#3672F5", "#7ACE5D"]
gradient_colors = [ "#36CE8A", '#7436F5']
color_palette_custom  = sns.set_palette(custom_colors)
theme_color = sns.color_palette(color_palette_custom, 9)
cmap_theme = LinearSegmentedColormap.from_list('custom_colormap', gradient_colors)
theme_color

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of available CPU cores: 8
Setting num_workers to: 7


[(0.21176470588235294, 0.807843137254902, 0.5411764705882353),
 (0.4549019607843137, 0.21176470588235294, 0.9607843137254902),
 (0.21568627450980393, 0.21176470588235294, 0.9568627450980393),
 (0.21176470588235294, 0.6823529411764706, 0.9607843137254902),
 (0.7019607843137254, 0.21176470588235294, 0.9607843137254902),
 (0.9725490196078431, 0.08627450980392157, 0.3686274509803922),
 (0.21176470588235294, 0.4392156862745098, 0.6039215686274509),
 (0.21176470588235294, 0.4470588235294118, 0.9607843137254902),
 (0.47843137254901963, 0.807843137254902, 0.36470588235294116)]

We will be exploring the Roberta model and see if we could find any good performance

In [41]:
#@title Preprocesing the dataset
dataframe = pd.read_csv(URL_path)
dataframe_test_import = pd.read_csv(URL_test)
dataframe_test_labels = pd.read_csv(URL_test_labels)
print(dataframe.head())

"""
Test dataset preparation
"""
attributes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
new_attributes = attributes + ["non_toxic"]

dataframe_test_labels_filtered = dataframe_test_labels.loc[dataframe_test_labels[attributes].sum(axis=1) >= 0]
dataframe_test_labels_filtered.shape
dataframe_test = dataframe_test_labels_filtered.merge(dataframe_test_import, on='id')
dataframe_test = dataframe_test.drop(columns=["id"])

SAMPLE_PERCENTAGE_TEST = 1 #@param
dataframe_test=dataframe_test.sample(frac=SAMPLE_PERCENTAGE_TEST,random_state=42)

print(dataframe_test.shape)
print("dataframe_test")
print(dataframe_test.head())

# Use the attributes list in the condition
dataframe["non_toxic"] = np.where(dataframe[attributes].any(axis=1), 0, 1)
dataframe.head()

                                        comment_text  toxic  severe_toxic  \
0  ExplanationWhy the edits made under my usernam...      0             0   
1  D'aww! He matches this background colour I'm s...      0             0   
2  Hey man, I'm really not trying to edit war. It...      0             0   
3  "MoreI can't make any real suggestions on impr...      0             0   
4  You, sir, are my hero. Any chance you remember...      0             0   

   obscene  threat  insult  identity_hate  
0        0       0       0              0  
1        0       0       0              0  
2        0       0       0              0  
3        0       0       0              0  
4        0       0       0              0  
(63978, 7)
dataframe_test
       toxic  severe_toxic  obscene  threat  insult  identity_hate  \
51138      1             0        0       0       0              0   
40102      0             0        0       0       0              0   
144        0             0        0  

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,non_toxic
0,ExplanationWhy the edits made under my usernam...,0,0,0,0,0,0,1
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,1
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,1
3,"""MoreI can't make any real suggestions on impr...",0,0,0,0,0,0,1
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,1


Now we will split the dataset to understand whats happening

,toxic,severe_toxic,obscene,threat,insult
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
144785,0,0,0,0,0
144786,0,0,0,0,0
144787,0,0,0,0,0
144788,0,0,0,0,0


In [42]:
#@title Split the dataset
# drop the non toxic column
dataframe.drop(columns= ["non_toxic"], inplace= True)

# sample sisze
SAMPLE_PERCENTAGE = 1 #@param
dataframe_clean=dataframe.sample(frac=SAMPLE_PERCENTAGE,random_state=42)

# split
train_size = 0.8 #@param
train_dataset=dataframe_clean.sample(frac=train_size,random_state=42)
validation_dataset=dataframe_clean.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(dataframe_clean.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(validation_dataset.shape))



FULL Dataset: (144790, 7)
TRAIN Dataset: (115832, 7)
TEST Dataset: (28958, 7)


In [43]:
def reduce_size_data_train(dataset:pd.DataFrame, sample_name: str)->pd.DataFrame:
    """ reduce the size of the dataset """
    target_y =dataset.iloc[:,1:].sum()
    attributes_non_toxic = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    non_toxic = dataset.loc[dataset[attributes_non_toxic].sum(axis=1) == 0]
    toxic = dataset.loc[dataset[attributes_non_toxic].sum(axis=1) > 0]
    non_toxic = non_toxic.sample(n=target_y["toxic"], random_state=7)
    dataset = pd.concat([toxic,non_toxic]).reset_index(drop=True)
    print(f"samples for {sample_name}: {dataset.shape}")

    return dataset

train_dataset = reduce_size_data_train(train_dataset, "train_dataset")

def normalize_values(dataset:pd.DataFrame)-> torch.tensor:
    target_y = dataset.iloc[:,1:].sum()
    weights_list = target_y.tolist()
    weights = torch.tensor(weights_list, dtype=torch.float32)
    normalized_weights = weights / weights.sum()
    print(normalized_weights)

    return normalized_weights

normalized_weights = normalize_values(train_dataset)

samples for train_dataset: (23857, 7)
tensor([0.4361, 0.0428, 0.2414, 0.0141, 0.2248, 0.0409])


We reduced the dataset size from 100k to 26k, removing a significant number of rows. This reduction allows the model to improve both in terms of training time and performance.

# Create a dataset

We will be preparing the dataset for the importation!

In [44]:
#@title custom dataset import class
class CustomDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, comment_str_title: str,  tokenizer, attributes, max_token_len= 128, sample= None):

        assert isinstance(dataframe, pd.DataFrame), "dataframe needs to be a pandas DataFrame for this code to work"

        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len
        self.attributes = attributes
        self.sample = sample
        self._prepare_data()

        # column to select
        self.comment_str_title = comment_str_title

    def _prepare_data(self):
        """Prepare the data and imported"""
        # data = pd.read_csv(self.data_path)
        data = self.dataframe.copy()
        # add the non toxic column
        data["non_toxic"] = np.where(data[attributes].any(axis=1), 0, 1)
        # data["unhealthy"] = np.where(data["healthy"] == 1, 0, 1)

        # extract the sample with some attributes
        if self.sample is not None:
            toxic = data.loc[data[self.attributes].sum(axis=1) > 0]
            non_toxic = data.loc[data[self.attributes].sum(axis=1) == 0]
            # extract the sample from the healthy values
            non_toxic = non_toxic.sample(min(self.sample, len(non_toxic)), random_state=7)
            self.data = pd.concat([non_toxic, toxic])
        else:
            self.data = data

    def __len__(self):
        """Length of the dataset"""
        return (len(self.data))

    def __getitem__(self, index):
        """Get item from the dataset"""
        data_row = self.data.iloc[index]

        # item as a string
        comment = str(data_row[self.comment_str_title])
        # attributes_numbers = torch.FloatTensor(data_row.iloc[self.attributes])
        attributes_numbers = torch.FloatTensor(data_row[self.attributes].values.astype(float))

        # extract the tokens and prepared to apply it to the comment line
        tokens = self.tokenizer.encode_plus(
                    comment,
                    add_special_tokens = True,
                    max_length = self.max_token_len,
                    padding = 'max_length',
                    truncation = True, # cap the comments to a max length of tokens
                    return_tensors = 'pt', # return tensors
                    return_attention_mask = True,
                    return_token_type_ids = False, # this its not need it
                    )

        tokens_as_words = self.tokenizer.convert_ids_to_tokens(tokens.input_ids.flatten().tolist())

        return {'input_id': tokens.input_ids.flatten(),
                'attention_mask': tokens.attention_mask.flatten(),
                'labels': attributes_numbers,
                'labels_names': self.attributes,
                'text_tokens': tokens_as_words}


# Data loader

In [45]:
#@title Custom class data loader
class Data_loader_Custom(pl.LightningDataModule):
    def __init__(self, train_dataframe, val_dataframe , attributes, comment_str_title: str, tokenizer,model_name = 'roberta-base',   batch_size= 16, max_token_len= 128 , sample= None, num_workers= 4, prediction_count = False):
        super().__init__()
        self.train_dataframe = train_dataframe
        self.val_dataframe = val_dataframe
        self.batch_size = batch_size
        self.max_token_len = max_token_len
        self.comment_str_title = comment_str_title
        self.attributes = attributes
        self.sample = sample
        self.num_workers = num_workers
        self.tokenizer = tokenizer
        self.model_name = model_name
        self.prediction_count = prediction_count

        # check if exist
        self.has_train_data = train_dataframe is not None
        self.has_val_data = val_dataframe is not None

    def __len__(self):
        if self.has_train_data:
            return len(self.train_dataset)
        if self.has_val_data:
            return len(self.validation_dataset)

    def len_predictions(self):
        if self.has_val_data:
            return len(self.validation_dataset)

    # this its using the LightningDataModule for seting up
    def setup(self, stage= None):
        if stage in (None, 'fit'):
            # train set
            self.train_dataset = CustomDataset(
                                dataframe=  self.train_dataframe,
                                comment_str_title =  self.comment_str_title,
                                attributes =  self.attributes,
                                tokenizer =  self.tokenizer,
                                max_token_len =  self.max_token_len,
                                sample= self.sample)
            # validation set
            self.validation_dataset = CustomDataset(
                                dataframe=  self.val_dataframe,
                                comment_str_title =  self.comment_str_title,
                                attributes =  self.attributes,
                                tokenizer =  self.tokenizer,
                                max_token_len =  self.max_token_len,
                                sample= self.sample)
        if stage == 'predict':
            self.predict_dataset  = CustomDataset(
                                dataframe=  self.val_dataframe,
                                comment_str_title =  self.comment_str_title,
                                attributes =  self.attributes,
                                tokenizer =  self.tokenizer,
                                max_token_len =  self.max_token_len,
                                sample= self.sample)

    # dataload the the different sets
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers=  self.num_workers, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.validation_dataset, batch_size=self.batch_size, num_workers= self.num_workers, shuffle=False)

    def predict_dataloader(self):
        return DataLoader(self.predict_dataset, batch_size=self.batch_size, num_workers= self.num_workers, shuffle=False)


----------

# Load the models

In [46]:
#@title preparing the architecture of the 2 models
class Classifier_model_loader(nn.Module):
    def __init__(self, config: dict, loss_function=None):
        super().__init__()
        self.config = config
        self.pretrained_model = AutoModel.from_pretrained(config['model_name'], return_dict=True)
        self.hidden = nn.Linear(self.pretrained_model.config.hidden_size, self.pretrained_model.config.hidden_size)
        self.classification = nn.Linear(self.pretrained_model.config.hidden_size, self.config["n_labels"])
        self.dropout = nn.Dropout(config["dropout_vals"])

        # Use provided loss function, or default to BCEWithLogitsLoss
        if loss_function is None:
            self.loss_function = nn.BCEWithLogitsLoss(reduction='mean')
        else:
            self.loss_function = loss_function

    def forward(self, input_ids, attention_mask):
        output = self.pretrained_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_out = torch.mean(output.last_hidden_state, 1)
        pooled_output = self.hidden(pooled_out)
        pooled_output = self.dropout(pooled_output)
        pooled_output = torch.relu(pooled_output)
        logits = self.classification(pooled_output)

        return logits

    def compute_loss(self, predictions, targets):
        return self.loss_function(predictions, targets)


# Initialize the model with the same configuration
roberta_015 = Classifier_model_loader(config_models,
                                       loss_function=nn.BCEWithLogitsLoss(weight=normalized_weights))
roberta_015 = roberta_015.to(device)
roberta_015.load_state_dict(torch.load(os.path.join(URL_model_saved_015, "model_015.pth")))

print("Model loaded successfully!")
print("-------------------------")
print(roberta_015)


<ipython-input-46-f1148236834d>:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  roberta_015.load_state_dict(torch.load(os.path.join(URL_model_saved_015, "model_015.pth")))

Model loaded successfully!
-------------------------
Classifier_model_loader(
  (pretrained_model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in

# Predict the model
We have two models, 015 and 017, where we will experiment with varying thresholds to understand how adjusting them can help filter values effectively. By setting different thresholds for each category, we aim to optimize the results and achieve better performance.

In [136]:
#@title predictibility model
def classify_raw_comments(model, dataset_loader_class):
    model.eval()
    all_predictions = []
    input_id_list = []
    attention_mask_list = []
    text_tokens_list = []

    for batch in dataset_loader_class.predict_dataloader():
        input_ids = batch['input_id'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        text_tokens = batch["text_tokens"]
        # text_tokens_list.append(text_tokens)
        # if isinstance(text_tokens, list) and isinstance(text_tokens[0], list):
        #     text_tokens = [token for sublist in text_tokens for token in sublist]  # Flatten the list
        text_tokens_list.append(text_tokens)

        # Make the prediction
        with torch.no_grad():  # Disable gradient calculation for inference
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        # Sigmoid activation for multi-label classification
        batch_predictions = torch.sigmoid(outputs).cpu().numpy()

        # predictions
        all_predictions.extend(batch_predictions)
        input_id_list.extend(input_ids)
        attention_mask_list.extend(attention_mask)
    # Stack all predictions into a numpy array
    flatten_predictions = np.stack(all_predictions)

    return flatten_predictions, input_id_list, attention_mask_list, text_tokens_list

"""
create data loaders
"""
validation_dataloader = Data_loader_Custom(
                   train_dataframe = train_dataset,
                   val_dataframe = validation_dataset,
                   attributes = attributes,
                   comment_str_title = 'comment_text',
                   tokenizer = AutoTokenizer.from_pretrained(config_models["model_name"]),
                   batch_size= config_models["batch_size"],
                   max_token_len= 256 ,
                   sample= None,
                   num_workers= num_workers_colab,
                   prediction_count= False)

test_dataloader = Data_loader_Custom(
                   train_dataframe = train_dataset,
                   val_dataframe = dataframe_test,
                   attributes = attributes,
                   comment_str_title = 'comment_text',
                   tokenizer = AutoTokenizer.from_pretrained(config_models["model_name"]),
                   batch_size= config_models["batch_size"],
                   max_token_len= 256 ,
                   sample= None,
                   num_workers= num_workers_colab,
                   prediction_count= False)


validation_dataloader.setup(stage='predict')
predictions_object_val_15, input_val_15, attention_val_15, text_tokens_val_15 = classify_raw_comments(roberta_015, validation_dataloader)
print(predictions_object_val_15.shape)

test_dataloader.setup(stage='predict')
predictions_object_test_15, input_test_15, attention_test_15, text_tokens_test_15 = classify_raw_comments(roberta_015, test_dataloader)
print(predictions_object_test_15.shape)

(28958, 6)
(63978, 6)


## Review confusion matrix for the thresholds 10 ann 40%

In [179]:
def threshold_calculation(predictions_object, true_labels, class_thresholds):
    """
    Compute confusion matrices per class using class-specific thresholds.

    :param predictions_object: Numpy array with model predictions.
    :param true_labels: Numpy array with ground truth labels.
    :param class_thresholds: Dictionary with thresholds for each class.
    """
    confusion_matrices = []
    num_classes = true_labels.shape[1]  # Number of classes

    # Loop through each class and apply the specific threshold
    for i in range(num_classes):
        attribute_name = list(class_thresholds.keys())[i]
        threshold = class_thresholds.get(attribute_name, 0.5)  # Default to 0.5 if no threshold


        true_class = true_labels[:, i]
        pred_probs = predictions_object[:, i]
        pred_class = (pred_probs >= threshold).astype(int)
        if len(true_class) != len(pred_class):
            raise ValueError("Mismatch between lengths of true labels and predictions.")

        # Compute confusion matrix for this class
        cm = confusion_matrix(true_class, pred_class)
        confusion_matrices.append({
            "class": attribute_name,
            "threshold": threshold,
            "confusion_matrix": cm
        })

    # Print results
    print("========================")
    for result in confusion_matrices:
        print(f"Class: {result['class']} (Threshold: {result['threshold']:.2f})")
        print("Confusion Matrix:")
        print(result["confusion_matrix"])
        print()
    print("========================")


class_thresholds = {
    "toxic": 0.50,
    "severe_toxic": 0.10,
    "obscene": 0.50,
    "threat": 0.50,
    "insult": 0.50,
    "identity_hate": 0.10
}

# Example input: replace with actual data
true_labels_val_15 = np.array(validation_dataset[attributes].values)
threshold_calculation(predictions_object_val_15, true_labels_val_15, class_thresholds)

Class: toxic (Threshold: 0.50)
Confusion Matrix:
[[24530  1596]
 [  211  2621]]

Class: severe_toxic (Threshold: 0.10)
Confusion Matrix:
[[27579  1077]
 [   11   291]]

Class: obscene (Threshold: 0.50)
Confusion Matrix:
[[27107   293]
 [  293  1265]]

Class: threat (Threshold: 0.50)
Confusion Matrix:
[[28880     0]
 [   78     0]]

Class: insult (Threshold: 0.50)
Confusion Matrix:
[[27079   365]
 [  422  1092]]

Class: identity_hate (Threshold: 0.10)
Confusion Matrix:
[[26996  1726]
 [   59   177]]



In [180]:
from sklearn.metrics import confusion_matrix, classification_report

def threshold_classification_report(predictions_object, true_labels, class_thresholds):
    """
    Evaluate predictions across multiple thresholds in a classification report.

    :param predictions_object: Numpy array with model predictions.
    :param true_labels: Numpy array with ground truth labels.
    :param class_thresholds: Dictionary with thresholds for each class.
    """
    # Initialize list to store confusion matrices and reports
    results = []

    # Loop through each class-specific threshold
    for i, class_name in enumerate(class_thresholds.keys()):
        class_threshold = class_thresholds[class_name]

        # Apply class-specific threshold to predictions for this class
        class_predicted_labels = (predictions_object[:, i] >= class_threshold).astype(int)

        # Generate confusion matrix for the current class
        class_conf_matrix = confusion_matrix(true_labels[:, i], class_predicted_labels)

        # Generate classification report for the current class
        class_report = classification_report(
            true_labels[:, i],
            class_predicted_labels,
            target_names=[f"Not {class_name}", class_name]
        )

        # Store results
        results.append({
            "class": class_name,
            "threshold": class_threshold,
            "confusion_matrix": class_conf_matrix,
            "classification_report": class_report
        })

    # Print results after all computations are done
    print("=" * 50)
    print("=" * 50)
    for result in results:
        print(f"Class: {result['class']} (Threshold: {result['threshold']:.2f})")
        print("Confusion Matrix:")
        print(result["confusion_matrix"])
        print("\nClassification Report:")
        print(result["classification_report"])
        print("========================")



# Example input: replace with actual data
true_labels_val_15 = np.array(validation_dataset[attributes].values)
threshold_classification_report(predictions_object_val_15, true_labels_val_15, class_thresholds)


Class: toxic (Threshold: 0.50)
Confusion Matrix:
[[24530  1596]
 [  211  2621]]

Classification Report:
              precision    recall  f1-score   support

   Not toxic       0.99      0.94      0.96     26126
       toxic       0.62      0.93      0.74      2832

    accuracy                           0.94     28958
   macro avg       0.81      0.93      0.85     28958
weighted avg       0.96      0.94      0.94     28958

Class: severe_toxic (Threshold: 0.10)
Confusion Matrix:
[[27579  1077]
 [   11   291]]

Classification Report:
                  precision    recall  f1-score   support

Not severe_toxic       1.00      0.96      0.98     28656
    severe_toxic       0.21      0.96      0.35       302

        accuracy                           0.96     28958
       macro avg       0.61      0.96      0.66     28958
    weighted avg       0.99      0.96      0.97     28958

Class: obscene (Threshold: 0.50)
Confusion Matrix:
[[27107   293]
 [  293  1265]]

Classification Report:
 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**After several tests, we have decided to stick with the following parameters for the model:**

```python
class_thresholds = {
    "toxic": 0.50,
    "severe_toxic": 0.10,
    "obscene": 0.50,
    "threat": 0.50,
    "insult": 0.50,
    "identity_hate": 0.10
}
```

These thresholds seem to yield the best performance so far. To validate this, we will apply these parameters to the test dataset and evaluate if there is an improvement in the results.


In [181]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

def all_threshold_classification_report(predictions_object, true_labels, class_thresholds):
    """
    Evaluate predictions across multiple thresholds in a classification report.

    :param predictions_object: Numpy array with model predictions.
    :param true_labels: Numpy array with ground truth labels.
    :param class_thresholds: Dictionary with thresholds for each class.
    """
    # Initialize lists to store predictions and true labels for all instances
    all_true_labels = []
    all_predicted_labels = []

    # Loop through each class-specific threshold and collect results
    for i, class_name in enumerate(class_thresholds.keys()):
        class_threshold = class_thresholds[class_name]

        # Apply class-specific threshold to predictions for this class
        class_predicted_labels = (predictions_object[:, i] >= class_threshold).astype(int)

        # Append to the lists for overall confusion matrix and classification report
        all_true_labels.extend(true_labels[:, i])
        all_predicted_labels.extend(class_predicted_labels)

    # Convert lists to numpy arrays
    all_true_labels = np.array(all_true_labels)
    all_predicted_labels = np.array(all_predicted_labels)

    # Compute confusion matrix for the entire set of predictions
    global_conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

    # Compute the classification report for the entire set of predictions
    global_class_report = classification_report(all_true_labels, all_predicted_labels)

    # Print results
    print("=" * 50)
    print("Global Confusion Matrix:")
    print(global_conf_matrix)
    print("=" * 50)
    print("\nGlobal Classification Report:")
    print(global_class_report)
    print("=" * 50)


# Example input: replace with actual data
true_labels_val_15 = np.array(validation_dataset[attributes].values)
all_threshold_classification_report(predictions_object_val_15, true_labels_val_15, class_thresholds)


Global Confusion Matrix:
[[162171   5057]
 [  1074   5446]]

Global Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.97      0.98    167228
           1       0.52      0.84      0.64      6520

    accuracy                           0.96    173748
   macro avg       0.76      0.90      0.81    173748
weighted avg       0.98      0.96      0.97    173748



Seems that after the tunning of the threshold we win a 85% of the values so this its a good result overall.

# test dataset prediction

In [182]:
# Example input: replace with actual data
true_labels_test_15 = np.array(dataframe_test[attributes].values)
threshold_calculation(predictions_object_test_15, true_labels_test_15, class_thresholds)

Class: toxic (Threshold: 0.50)
Confusion Matrix:
[[50786  7102]
 [  422  5668]]

Class: severe_toxic (Threshold: 0.10)
Confusion Matrix:
[[59726  3885]
 [   34   333]]

Class: obscene (Threshold: 0.50)
Confusion Matrix:
[[59029  1258]
 [ 1109  2582]]

Class: threat (Threshold: 0.50)
Confusion Matrix:
[[63724    43]
 [  211     0]]

Class: insult (Threshold: 0.50)
Confusion Matrix:
[[59665   886]
 [ 1475  1952]]

Class: identity_hate (Threshold: 0.10)
Confusion Matrix:
[[57153  6113]
 [  199   513]]



In [183]:
threshold_classification_report(predictions_object_test_15, true_labels_test_15, class_thresholds)

Class: toxic (Threshold: 0.50)
Confusion Matrix:
[[50786  7102]
 [  422  5668]]

Classification Report:
              precision    recall  f1-score   support

   Not toxic       0.99      0.88      0.93     57888
       toxic       0.44      0.93      0.60      6090

    accuracy                           0.88     63978
   macro avg       0.72      0.90      0.77     63978
weighted avg       0.94      0.88      0.90     63978

Class: severe_toxic (Threshold: 0.10)
Confusion Matrix:
[[59726  3885]
 [   34   333]]

Classification Report:
                  precision    recall  f1-score   support

Not severe_toxic       1.00      0.94      0.97     63611
    severe_toxic       0.08      0.91      0.15       367

        accuracy                           0.94     63978
       macro avg       0.54      0.92      0.56     63978
    weighted avg       0.99      0.94      0.96     63978

Class: obscene (Threshold: 0.50)
Confusion Matrix:
[[59029  1258]
 [ 1109  2582]]

Classification Report:
 

In [185]:
all_threshold_classification_report(predictions_object_test_15, true_labels_test_15, class_thresholds)

Global Confusion Matrix:
[[350083  19287]
 [  3450  11048]]

Global Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.95      0.97    369370
           1       0.36      0.76      0.49     14498

    accuracy                           0.94    383868
   macro avg       0.68      0.85      0.73    383868
weighted avg       0.97      0.94      0.95    383868



It appears that the model's performance has improved compared to previous runs. We've achieved a 15% increase in recall, which is the metric we prioritized for evaluation. Additionally, we are still exceeding expectations in terms of accuracy.


In [186]:
import itertools

df = dataframe.iloc[:,1:-1]
columns = df.columns
combinations = list(itertools.combinations(columns, 2))
combination_counts = []
for combo in combinations:
    combo_df = df[list(combo)]
    combo_count = combo_df.value_counts().reset_index(name='count')
    for _, row in combo_count.iterrows():
        combination_counts.append({
            "combination": combo,
            "values": row[list(combo)].to_list(),
            "count": row["count"]
        })

combination_counts_df = pd.DataFrame(combination_counts)
print(combination_counts_df)


                combination  values   count
0     (toxic, severe_toxic)  [0, 0]  130364
1     (toxic, severe_toxic)  [1, 0]   12987
2     (toxic, severe_toxic)  [1, 1]    1439
3          (toxic, obscene)  [0, 0]  129907
4          (toxic, obscene)  [1, 1]    7519
5          (toxic, obscene)  [1, 0]    6907
6          (toxic, obscene)  [0, 1]     457
7           (toxic, threat)  [0, 0]  130337
8           (toxic, threat)  [1, 0]   13999
9           (toxic, threat)  [1, 1]     427
10          (toxic, threat)  [0, 1]      27
11          (toxic, insult)  [0, 0]  129870
12          (toxic, insult)  [1, 0]    7430
13          (toxic, insult)  [1, 1]    6996
14          (toxic, insult)  [0, 1]     494
15  (severe_toxic, obscene)  [0, 0]  136755
16  (severe_toxic, obscene)  [0, 1]    6596
17  (severe_toxic, obscene)  [1, 1]    1380
18  (severe_toxic, obscene)  [1, 0]      59
19   (severe_toxic, threat)  [0, 0]  142998
20   (severe_toxic, threat)  [1, 0]    1338
21   (severe_toxic, threat)  [0,

It also seems that some categories may not be predictably classified, as they are not connected in any way.

# Summary


Tuning the thresholds has provided a slight improvement in the model, helping us understand that adjusting them can lead to better performance across different categories. For now, we will stick with this approach.